# 1. Set up packages

In [ ]:
import os
import sys
from pathlib import Path
import subprocess

# Helper to run shell commands
def run(cmd, check=True):
    print(f"$ {cmd}")
    with open("output.log", "w") as out_file, open("error.log", "w") as err_file:
        proc = subprocess.run(cmd, shell=True, stdout=out_file, stderr=err_file, text=True)
        print(proc.stdout)
        if proc.returncode != 0:
            print(proc.stderr)
            if check:
                raise RuntimeError(f"Command failed: {cmd}")
    return proc


# 2. Download and compress fastq files from two timepoints from SRA

In [ ]:
# Download fastq files from SRA using fasterq-dump
run("fasterq-dump --split-files SRR13973794")
run("fasterq-dump --split-files SRR13973795")

# Compress fastq files to reduce file size
run("gzip SRR1397379*.fastq")


# 3. Run FastQC to check sequence quality

In [ ]:
# Run FastQC on raw reads
run("mkdir fastqc_raw")
run("fastqc -o fastqc_raw SRR13973794_1.fastq.gz SRR13973794_2.fastq.gz")
run("fastqc -o fastqc_raw SRR13973795_1.fastq.gz SRR13973795_2.fastq.gz")

# 4. Trimming - fastp

In [ ]:
# Trimming adapter and short sequences using fastp
run("fastp -i SRR13973794_1.fastq.gz \
    -I SRR13973794_2.fastq.gz \
    -o SRR13973794_1.trim.fastq.gz \
    -O SRR13973794_2.trim.fastq.gz \
    --detect_adapter_for_pe \
    --trim_poly_g \
    --thread 8 \
    -j SRR13973794_fastp.json -h SRR13973794_fastp.html")

run("fastp -i SRR13973795_1.fastq.gz \
    -I SRR13973795_2.fastq.gz \
    -o SRR13973795_1.trim.fastq.gz \
    -O SRR13973795_2.trim.fastq.gz \
    --detect_adapter_for_pe \
    --trim_poly_g \
    --thread 8 \
    -j SRR13973795_fastp.json -h SRR13973795_fastp.html")

# 5. Rerun FastQC after trimming

In [ ]:
# 5. Rerun FastQC after trimming to confirm quality
run("mkdir fastqc_trimmed")
run("fastqc -o fastqc_trimmed SRR13973794_1.trim.fastq.gz SRR13973794_2.trim.fastq.gz")
run("fastqc -o fastqc_trimmed SRR13973795_1.trim.fastq.gz SRR13973795_2.trim.fastq.gz")

# 6. Reference preparation - bowtie2, samtools, picard

In [ ]:
# Indexing reference genome and creating .fai and .dict files that will be needed later
### Only need to run once
#run("bowtie2-build Homo_sapiens_assembly38.fasta Homo_sapiens_assembly38.fasta --threads 8")
#run("samtools faidx Homo_sapiens_assembly38.fasta")
#run("picard CreateSequenceDictionary R=Homo_sapiens_assembly38.fasta O=Homo_sapiens_assembly38.dict")

# 7. Alignment - bowtie2

In [ ]:
# Run bowtie 2 to align reads for both samples
run('bowtie2 --rg-id plasma_99 \
--rg SM:plasma_99 \
--rg PL:ILLUMINA \
--rg LB:Targeted-Capture \
--rg PU:HiSeq2500 \
--rg DS:"Strategy=Targeted-Capture;Source=GENOMIC;Selection=Hybrid-Selection" \
-p 8 \
-x Homo_sapiens_assembly38.fasta \
-1 SRR13973794_1.trim.fastq.gz \
-2 SRR13973794_2.trim.fastq.gz \
-S SRR13973794_aligned_reads.sam')

run('bowtie2 --rg-id plasma_98 \
--rg SM:plasma_98 \
--rg PL:ILLUMINA \
--rg LB:Targeted-Capture \
--rg PU:HiSeq2500 \
--rg DS:"Strategy=Targeted-Capture;Source=GENOMIC;Selection=Hybrid-Selection" \
-p 8 \
-x Homo_sapiens_assembly38.fasta \
-1 SRR13973795_1.trim.fastq.gz \
-2 SRR13973795_2.trim.fastq.gz \
-S SRR13973795_aligned_reads.sam')

# 8. Convert SAM files to BAM - samtools

In [ ]:
# Convert aligned SAM files to BAM
run("samtools view -bS SRR13973794_aligned_reads.sam > SRR13973794_aligned_reads.bam")
run("samtools view -bS SRR13973795_aligned_reads.sam > SRR13973795_aligned_reads.bam")

# 9. Extract APC reads - samtools

In [ ]:
# Extract only reads that overlap APC gene

# Using grep to get coordinates for GTF file
run("grep ENSG00000134982 gencode.v47.basic.protein_coding.canonical.gtf") # APC gene coords. = chr5:112707518-112846239

# Using samtools to extract reads from APC
# Sort reads first
run("samtools sort -o SRR13973794_aligned_reads.sorted.bam SRR13973794_aligned_reads.bam")
run("samtools sort -o SRR13973795_aligned_reads.sorted.bam SRR13973795_aligned_reads.bam")

# index sorted bam file
run("samtools index SRR13973794_aligned_reads.sorted.bam")
run("samtools index SRR13973795_aligned_reads.sorted.bam")

# Select only reads overlaping the APC gene location
run("samtools view -b SRR13973794_aligned_reads.sorted.bam chr5:112707518-112846239 > SRR13973794_APC_reads.bam")
run("samtools view -b SRR13973795_aligned_reads.sorted.bam chr5:112707518-112846239 > SRR13973795_APC_reads.bam")



# 10. Mark duplicates - MarkDuplicatesSpark

In [ ]:
# Mark duplicates in aligned read BAM files
run("gatk MarkDuplicatesSpark \
    -I SRR13973794_APC_reads.bam \
    -O SRR13973794_APC_marked_duplicates.bam \
    -M SRR13973794_APC_marked_dup_metrics.txt")
    
run("gatk MarkDuplicatesSpark \
    -I SRR13973795_APC_reads.bam \
    -O SRR13973795_APC_marked_duplicates.bam \
    -M SRR13973795_APC_marked_dup_metrics.txt")
    

# 11. Base score recalibration - GATK

In [ ]:
# Use GATK for base score recalibration based on known hg38 indels sites
run("gatk BaseRecalibrator \
  -I SRR13973794_APC_marked_duplicates.bam \
  -R Homo_sapiens_assembly38.fasta \
  --known-sites Mills_and_1000G_gold_standard.indels.hg38.vcf.gz \
  -O SRR13973794_APC_marked_duplicates_recal_data.table")
  
run("gatk ApplyBQSR \
    -R Homo_sapiens_assembly38.fasta \
    -I SRR13973794_APC_marked_duplicates.bam \
    --bqsr-recal-file SRR13973794_APC_marked_duplicates_recal_data.table \
    -O SRR13973794_APC_marked_duplicates_recal_data.bam")

run("gatk BaseRecalibrator \
  -I SRR13973795_APC_marked_duplicates.bam \
  -R Homo_sapiens_assembly38.fasta \
  --known-sites Mills_and_1000G_gold_standard.indels.hg38.vcf.gz \
  -O SRR13973795_APC_aligned_reads_recal_data.table")
  
run("gatk ApplyBQSR \
    -R Homo_sapiens_assembly38.fasta \
    -I SRR13973795_APC_marked_duplicates.bam \
    --bqsr-recal-file SRR13973795_aligned_reads_recal_data.table \
    -O SRR13973795_APC_marked_duplicates_recal_data.bam")

# 12. Create mpilup files - samtools

In [ ]:
# Use samtools to convert BAM files to mpileup format
run("samtools mpileup -f Homo_sapiens_assembly38.fasta SRR13973794_APC_marked_duplicates_recal_data.bam > SRR13973794_APC.pileup")
run("samtools mpileup -f Homo_sapiens_assembly38.fasta SRR13973795_APC_marked_duplicates_recal_data.bam > SRR13973795_APC.pileup")

# 13. Call variants - VarScan2

In [ ]:
# Run VarScan2 to call SNP variants
run("java -jar ./VarScan.v2.3.9.jar mpileup2snp SRR13973794_APC.pileup --min-coverage 10 --min-avg-qual 20 --min-var-freq 0.20 --p-value 0.05 --output-vcf 1 > SRR13973794_APC.snp.vcf")
run("java -jar ./VarScan.v2.3.9.jar mpileup2snp SRR13973795_APC.pileup --min-coverage 10 --min-avg-qual 20 --min-var-freq 0.20 --p-value 0.05 --output-vcf 1 > SRR13973795_APC.snp.vcf")

# Compare pre and post treatment
run("ava -jar VarScan.v2.3.9.jar somatic SRR13973794_APC.pileup SRR13973795_APC.pileup --min-coverage 10 --min-var-freq 0.20 --p-value 0.05 --somatic-p-value 0.05 --output-vcf 1 > comp_APC.snp.vcf")

# 17 SNPs in pre-treatment samples and 21 SNPs in during treatment sample, shows that there are new somatic mutations that have appeared
# and the tumor is likely evolving and potentially developing resistance to the current treatment



# 14. Annotate variants - snpEff and snpSift

In [ ]:
### NOTE: Need JAVA 21+ to be able to run snpEff and snpSift
# Use snpEff to annotate the gene variants
run("java -Xmx4G -jar ./snpEff/snpEff.jar eff -v GRCh38.86 SRR13973794_APC.snp.vcf > pre_treatment_APC_calls.filtered.vcf")
run("java -Xmx4G -jar ./snpEff/snpEff.jar eff -v GRCh38.86 SRR13973795_APC.snp.vcf > during_treatment_APC_calls.filtered.vcf")

# Use snpSift to filter for only variants that passed snpEff filter and add ClinVAR designations to the annotated gene variants
!java -jar ./snpEff/SnpSift.jar filter "(FILTER = 'PASS')" pre_treatment_APC_calls.filtered.vcf |java -jar ./snpEff/SnpSift.jar annotate clinvar_20251109.vcf.gz > pre_treatment_APC_annotated.vcf
!java -jar ./snpEff/SnpSift.jar filter "(FILTER = 'PASS')" post_treatment_APC_calls.filtered.vcf |java -jar ./snpEff/SnpSift.jar annotate clinvar_20251109.vcf.gz > during_treatment_APC_annotated.vcf

## Do not see any previously described clinical variants in the APC gene for this patient. Could look at other clinical annotation
## datasets such as COSMIC

# 15. Next steps

In [ ]:
# Create visualization of APC gene with locations of the SNPs
# Run all samples and see if trend towards increased SNPs in APC holds true and if SNPs occure in the same places
# Create a simple plot of APC SNPs before and during/after treatment with anti-EGFR drugs